## Homogeneous Gas

Here is a notebook for homogeneous gas model.

Here we are talking about a homogeneous gas bulk of neutrinos with single energy. The EoM is
$$
i \partial_t \rho_E = \left[ \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F \int_0^\infty dE' ( \rho_{E'} - \bar \rho_{E'} ) ,\rho_E \right]
$$

while the EoM for antineutrinos is
$$
i \partial_t \bar\rho_E = \left[- \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F \int_0^\infty dE' ( \rho_{E'} - \bar \rho_{E'} ) ,\bar\rho_E \right]
$$

Initial:
Homogeneous, Isotropic, Monoenergetic $\nu_e$ and $\bar\nu_e$

The equations becomes
$$
i \partial_t \rho_E = \left[ \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F   ( \rho_{E} - \bar \rho_{E} ) ,\rho_E \right]
$$
$$
i \partial_t \bar\rho_E = \left[- \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F ( \rho_{E} - \bar \rho_{E} ) ,\bar\rho_E \right]
$$

where

$$
B = \frac{1}{2} \begin{pmatrix} 
-\cos 2\theta_v & \sin 2\theta_v \\
\sin 2\theta_v & \cos 2\theta_v
\end{pmatrix} = 
\begin{pmatrix} 
-0.38729833462 & 0.31622776601\\
0.31622776601 & 0.38729833462
\end{pmatrix}
$$

$$
L = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

Initial condition 
$$
\rho(t=0) = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

$$
\bar\rho(t=0) =\begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

define the following quantities

1. hbar$=\hbar$
2. delm2E$= \delta m^2/2E$
3. lamb $= \lambda$
4. gF $= G_F$

## Numerical

In [138]:
# This line configures matplotlib to show figures embedded in the notebook, 
# instead of opening a new window for each figure. More about that later. 
# If you are using an old version of IPython, try using '%pylab inline' instead.
%matplotlib inline
%load_ext snakeviz

import numpy as np
from scipy.optimize import minimize
from scipy.special import expit
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D

import timeit

import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [164]:
# hbar=1.054571726*10**(-34)
hbar=1
delm2E=1
lamb=1
gF=1

## Here are some matrices to be used

elM = np.array([[1,0],[0,0]])
bM = 1/2*np.array( [ [ - 0.38729833462,0.31622776601] , [0.31622776601,0.38729833462] ] )

## sqareroot of 2
sqrt2=np.sqrt(2)

Using Mathematica, I can find the 4\*2 equations

In [165]:

#r11prime(t)

## The matrix eqn for neutrinos. Symplify the equation to the form A.X=0. Here I am only writing down the LHS.
## Eqn for r11'
# 1/2*( r21(t)*( bM12*delm2E - 2*sqrt2*gF*rb12(t) ) + r12(t) * ( -bM21*delm2E + 2*sqrt2*gF*rb21(t) ) - 1j*r11prime(t)  )
## Eqn for r12'
# 1/2*( r22(t)* ( bM12 ) )

### wait a minute I don't actually need to write down this. I can just do this part in numpy.

I am going to substitute all density matrix elements using their corrosponding network expressions.

So first of all, I need the network expression for the unknown functions.

A function is written as

$$ y_i= 1+t_i v_k f(t_i w_k+u_k) ,$$

while it's derivative is

$$v_k f(t w_k+u_k) + t v_k f(tw_k+u_k) (1-f(tw_k+u_k)) w_k .$$

Now I can write down the equations using these two forms.

In [166]:
def trigf(x):
    #return 1/(1+np.exp(-x)) # It's not bad to define this function here for people could use other functions other than expit(x).
    return expit(x)

In [167]:
## The time derivative part

### Here are the initial conditions

init = np.array( [[1,0],[0,1]] )
initb = np.array([[0,0],[0,0]])

### For neutrinos

def rho(x,ti,initialCondition): # x is the input structure arrays, ti is a time point

    v11,w11,u11,v12,w12,u12,v21,w21,u21,v22,w22,u22 = np.split(x,12)[:12]
        
    elem11= np.sum(ti * v11 * trigf( ti*w11 +u11 ) )
    elem12= np.sum(ti * v12 * trigf( ti*w12 +u12 ) )
    elem21= np.sum(ti * v21 * trigf( ti*w21 +u21 ) )
    elem22= np.sum(ti * v22 * trigf( ti*w22 +u22 ) )
    
    return initialCondition + np.array([[ elem11 , elem12 ],[elem21, elem22]])

def rhob(xb,ti,initialConditionb): # x is the input structure arrays, ti is a time point

    vb11,wb11,ub11,vb12,wb12,ub12,vb21,wb21,ub21,vb22,wb22,ub22 = np.split(xb,12)[:12]

    elem11= np.sum(ti * vb11 * trigf( ti*wb11 +ub11 ) )
    elem12= np.sum(ti * vb12 * trigf( ti*wb12 +ub12 ) )
    elem21= np.sum(ti * vb21 * trigf( ti*wb21 +ub21 ) )
    elem22= np.sum(ti * vb22 * trigf( ti*wb22 +ub22 ) )
    
    return initialConditionb + np.array([[ elem11 , elem12 ],[elem21, elem22]])



In [168]:
## Test
xtemp=np.ones(120)
rho(xtemp,1,init)

array([[ 9.80797078,  8.80797078],
       [ 8.80797078,  9.80797078]])

In [169]:
## Define Hamiltonians for both

def hamil(x,xb,ti,initialCondition,initialConditionb):
    
    return delm2E*bM + lamb*elM + sqrt2*gF*( rho(x,ti,initialCondition) - rhob(xb,ti,initialConditionb) )


def hamilb(x,xb,ti,initialCondition,initialConditionb):
    
    return -delm2E*bM + lamb*elM + sqrt2*gF*( rho(x,ti,initialCondition) - rhob(xb,ti,initialConditionb) )

In [170]:
## The commutator

def comm(x,xb,ti,initialCondition,initialConditionb):
    
    return np.dot(hamil(x,xb,ti,initialCondition,initialConditionb), rho(x,ti,initialCondition) ) - np.dot(rho(x,ti,initialCondition), hamil(x,xb,ti,initialCondition,initialConditionb) )

def commb(x,xb,ti,initialCondition,initialConditionb):
    
    return np.dot(hamilb(x,xb,ti,initialCondition,initialConditionb), rhob(xb,ti,initialConditionb) ) - np.dot(rhob(xb,ti,initialConditionb), hamilb(x,xb,ti,initialCondition,initialConditionb) )


In [171]:
## Test

print "neutrino\n",comm(xtemp,xtemp,1,init,initb)
print "antineutrino\n",commb(xtemp,xtemp,0.5,init,initb)

neutrino
[[ 0.          8.80797078]
 [-8.80797078  0.        ]]
antineutrino
[[ 0.          4.08787238]
 [-4.08787238  0.        ]]


In [172]:
## The COST of the eqn set

def costTi(x,xb,ti,initialCondition,initialConditionb):
    
    v11,w11,u11,v12,w12,u12,v21,w21,u21,v22,w22,u22 = np.split(x,12)[:12]
    vb11,wb11,ub11,vb12,wb12,ub12,vb21,wb21,ub21,vb22,wb22,ub22 = np.split(xb,12)[:12]
    
    fvec11 = np.array(trigf(ti*w11 + u11) )  # This is a vector!!!
    fvec12 = np.array(trigf(ti*w12 + u12) )
    fvec21 = np.array(trigf(ti*w21 + u21) )
    fvec22 = np.array(trigf(ti*w22 + u22) )
    
    fvecb11 = np.array(trigf(ti*wb11 + ub11) )  # This is a vector!!!
    fvecb12 = np.array(trigf(ti*wb12 + ub12) )
    fvecb21 = np.array(trigf(ti*wb21 + ub21) )
    fvecb22 = np.array(trigf(ti*wb22 + ub22) )
    
    
    costi11= ( np.sum (v11*fvec11 + ti * v11* fvec11 * ( 1 -  fvec11  ) * w11 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[0,0] )  )  
    costi12= ( np.sum (v12*fvec12 + ti * v12* fvec12 * ( 1 -  fvec12  ) * w12 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[0,1] )  )  
    costi21= ( np.sum (v21*fvec21 + ti * v21* fvec21 * ( 1 -  fvec21  ) * w21 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[1,0] )  )  
    costi22= ( np.sum (v22*fvec22 + ti * v22* fvec22 * ( 1 -  fvec22  ) * w22 ) + 1j*  ( comm(x,xb,ti,initialCondition,initialConditionb)[1,1] )  )  

    costbi11= ( np.sum (vb11*fvecb11 + ti * vb11* fvecb11 * ( 1 -  fvecb11  ) * wb11 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[0,0] )  )  
    costbi12= ( np.sum (vb12*fvecb12 + ti * vb12* fvecb12 * ( 1 -  fvecb12  ) * wb12 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[0,1] )  )  
    costbi21= ( np.sum (vb21*fvecb21 + ti * vb21* fvecb21 * ( 1 -  fvecb21  ) * wb21 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[1,0] )  )  
    costbi22= ( np.sum (vb22*fvecb22 + ti * vb22* fvecb22 * ( 1 -  fvecb22  ) * wb22 ) + 1j*  ( commb(x,xb,ti,initialCondition,initialConditionb)[1,1] )  )  
    
    return (np.real(costi11))**2 + (np.real(costi12))**2+ (np.real(costi21))**2 +  (np.real(costi22))**2 + (np.real(costbi11))**2 + (np.real(costbi12))**2 +(np.real(costbi21))**2 + (np.real(costbi22))**2 + (np.imag(costi11))**2 + (np.imag(costi12))**2+ (np.imag(costi21))**2 +  (np.imag(costi22))**2 + (np.imag(costbi11))**2 + (np.imag(costbi12))**2 +(np.imag(costbi21))**2 + (np.imag(costbi22))**2    


In [173]:
costTi(xtemp,xtemp,0,init,initb)

427.55731631081869

In [174]:
## Calculate the total cost

def cost(xtot,t,initialCondition,initialConditionb):
    
    x,xb = np.split(xtot,2)[:2]

    t = np.array(t)
    
    costTotal = np.sum( costTList(x,xb,t,initialCondition,initialConditionb)  )
        
    return costTotal
    

def costTList(x,xb,t,initialCondition,initialConditionb):  ## This is the function WITHOUT the square!!! 
        
    t = np.array(t)
    
    costList = np.asarray([])
    
    for temp in t:
        tempElement = costTi(x,xb,temp,initialCondition,initialConditionb)
        costList = np.append(costList, tempElement)
        
    return np.array(costList)

    

In [175]:
ttemp = np.linspace(0,10)
print ttemp

[  0.           0.20408163   0.40816327   0.6122449    0.81632653
   1.02040816   1.2244898    1.42857143   1.63265306   1.83673469
   2.04081633   2.24489796   2.44897959   2.65306122   2.85714286
   3.06122449   3.26530612   3.46938776   3.67346939   3.87755102
   4.08163265   4.28571429   4.48979592   4.69387755   4.89795918
   5.10204082   5.30612245   5.51020408   5.71428571   5.91836735
   6.12244898   6.32653061   6.53061224   6.73469388   6.93877551
   7.14285714   7.34693878   7.55102041   7.75510204   7.95918367
   8.16326531   8.36734694   8.57142857   8.7755102    8.97959184
   9.18367347   9.3877551    9.59183673   9.79591837  10.        ]


In [176]:
ttemp = np.linspace(0,10)
print costTList(xtemp,xtemp,ttemp,init,initb)
print cost(xtemp,ttemp,init,initb)

[   427.55731631    528.89091088    645.65674303    779.2781299
    931.92977081   1106.23040979   1304.89968942   1530.47431911
   1785.12578983   2070.57984981   2388.11501175   2738.61081605
   3122.61989655   3540.44514969   3992.21073277   4477.92149059
   4997.50934606   5550.86745364   6137.87397501   6758.40764085
   7412.35715103   8099.6261654    8820.13528321   9573.82207266
  10360.63992401  11180.55626939  12033.5505356   12919.61206652
  13838.73815861  14790.93228828  15776.20256699  16794.56043168
  17846.01956113  18930.59499911  20048.30246054  21199.15779534
  22383.17658549  23600.37385222  24850.76385291  26134.35994965
  27451.17453421  28801.21899636  30184.50372493  31601.03813271
  33050.83069836  34533.88901945  36050.21987249  37599.82927646
  39182.7225573   40798.9044115 ]
177665.754409


## Minimization

Here is the minimization

In [177]:
tlin = np.linspace(0,5,11)
initGuess = np.ones(120)
# initGuess = np.random.rand(1,30)+2

costF = lambda x: cost(x,tlin,init,initb)

In [178]:
cost(initGuess,tlin,init,initb)

11529.167843869518

In [162]:
## %%snakeviz
# startCG = timeit.default_timer()
#costFResultCG = minimize(costF,initGuess,method="CG")
#stopCG = timeit.default_timer()

#print stopCG - startCG

#print costFResultCG

In [179]:
%%snakeviz
startSLSQP = timeit.default_timer()
costFResultSLSQP = minimize(costF,initGuess,method="SLSQP")
stopSLSQP = timeit.default_timer()

print stopSLSQP - startSLSQP

print costFResultSLSQP

1380.02428007
  status: 0
 success: True
    njev: 50
    nfev: 6138
     fun: 3.1109570709063641e-06
       x: array([ -5.51671331e-04,   3.97304675e-04,   2.38604217e-02,
         2.60967024e-02,  -4.83801927e-02,   1.32875718e-01,
         1.32008368e-01,   1.19856942e-01,   1.23806872e-01,
         1.83623988e-01,   1.64408094e+00,   1.64321746e+00,
         1.61430064e+00,   1.60774236e+00,   1.67969896e+00,
         1.24309138e-06,   1.24309138e-06,   1.12752996e-06,
         1.13101160e-06,   1.22975100e-06,   9.46320460e-01,
         9.46320460e-01,   9.46320528e-01,   9.46320527e-01,
         9.46320523e-01,   9.70285977e-01,   9.70285977e-01,
         9.70286082e-01,   9.70285957e-01,   9.70285619e-01,
         4.96520764e-07,   4.47604890e-07,   5.83269418e-07,
         5.83412503e-07,   5.37286054e-07,   8.63418445e-01,
         8.63418475e-01,   8.63418573e-01,   8.63418576e-01,
         8.63418574e-01,   9.46810045e-01,   9.46809998e-01,
         9.46810167e-01,   9.46810

In [181]:
costFResultSLSQP.get('x')

array([ -5.51671331e-04,   3.97304675e-04,   2.38604217e-02,
         2.60967024e-02,  -4.83801927e-02,   1.32875718e-01,
         1.32008368e-01,   1.19856942e-01,   1.23806872e-01,
         1.83623988e-01,   1.64408094e+00,   1.64321746e+00,
         1.61430064e+00,   1.60774236e+00,   1.67969896e+00,
         1.24309138e-06,   1.24309138e-06,   1.12752996e-06,
         1.13101160e-06,   1.22975100e-06,   9.46320460e-01,
         9.46320460e-01,   9.46320528e-01,   9.46320527e-01,
         9.46320523e-01,   9.70285977e-01,   9.70285977e-01,
         9.70286082e-01,   9.70285957e-01,   9.70285619e-01,
         4.96520764e-07,   4.47604890e-07,   5.83269418e-07,
         5.83412503e-07,   5.37286054e-07,   8.63418445e-01,
         8.63418475e-01,   8.63418573e-01,   8.63418576e-01,
         8.63418574e-01,   9.46810045e-01,   9.46809998e-01,
         9.46810167e-01,   9.46810100e-01,   9.46810113e-01,
         2.48299857e-06,   4.16500601e-06,   2.94448303e-06,
         2.74235494e-06,

In [183]:
np.savetxt('./assets/homogen/optimize_ResultSLSQP.txt', costFResultSLSQP.get('x'), delimiter = ',')

## Functions

Find the solutions to each elements.

In [200]:
# The first element of neutrino density matrix
xresult = np.split(costFResultSLSQP.get('x'),2)[0]
xresultb = np.split(costFResultSLSQP.get('x'),2)[1]
# print xresult11

pltdata11 = np.array([])
pltdata22 = np.array([])

for i in np.linspace(0,5,11):
    pltdata11 = np.append(pltdata11 ,rho(xresult,i,init)[0,0] )
    
print pltdata11

for i in np.linspace(0,5,11):
    pltdata22 = np.append(pltdata22 ,rho(xresult,i,init)[1,1] )
    
print pltdata22

print "----------------------------------------"

pltdatab11 = np.array([])
pltdatab22 = np.array([])

for i in np.linspace(0,5,11):
    pltdatab11 = np.append(pltdatab11 ,rho(xresultb,i,init)[0,0] )
    
print pltdatab11

for i in np.linspace(0,5,11):
    pltdatab22 = np.append(pltdatab22 ,rho(xresultb,i,init)[1,1] )
    
print pltdatab22

[ 1.          1.0002881   1.00043797  1.00047686  1.00042994  1.00032026
  1.00016859  0.99999345  0.99981112  0.99963572  0.99947933]
[ 1.          1.00000602  1.00001275  1.00001986  1.00002714  1.00003446
  1.00004177  1.00004905  1.00005629  1.00006349  1.00007066]
----------------------------------------
[ 1.          0.99999955  0.99999906  0.99999853  0.99999799  0.99999745
  0.9999969   0.99999636  0.99999582  0.99999529  0.99999475]
[ 1.          1.00025356  1.00034021  1.0002945   1.00015328  0.99995576
  0.99974352  0.99956048  0.99945297  0.99946961  0.99966131]


In [205]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rho11')
plt.plot(np.linspace(0,5,11),pltdata11,"b4-",label="rho11")
py.iplot_mpl(plt.gcf(),filename="HG-rho11")


# tls.embed("https://plot.ly/~emptymalei/73/")

In [204]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rho22')
plt.plot(np.linspace(0,5,11),pltdata22,"r4-",label="rho22")
py.iplot_mpl(plt.gcf(),filename="HG-rho22")

In [203]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rhob11')
plt.plot(np.linspace(0,5,11),pltdatab11,"b*-",label="rhob11")
py.iplot_mpl(plt.gcf(),filename="HG-rhob11")


In [202]:
plt.figure(figsize=(20,12.36))
plt.ylabel('Time')
plt.xlabel('rhob22')
plt.plot(np.linspace(0,5,11),pltdatab11,"b*-",label="rhob22")
py.iplot_mpl(plt.gcf(),filename="HG-rhob22")


## Practice

In [ ]:
xtemp1 = np.arange(4)
xtemp1.shape = (2,2)
print xtemp1
xtemp1[0,1]
np.dot(xtemp1,xtemp1)
xtemp1[0,1]